---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [0]:
# import numpy as np
# import pandas as pd
# from google.colab import files
# uploaded = files.upload()
# spam.csv

In [0]:
# import warnings
# warnings.filterwarnings('ignore', category=FutureWarning)

In [4]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [0]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

In [0]:
def answer_one():
      
    ans = len(spam_data[spam_data['target'] == 1]) / len(spam_data) * 100
    
    return ans

In [7]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [0]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    
    # convert the dataframe to list for the following processing 
    texts = spam_data['text'].T.tolist()

    # cv train 
    cv = CountVectorizer()
    cv_fit = cv.fit_transform(texts)
    
    # sort the words by length in order
    sortedwords = sorted(cv.get_feature_names(), key=len)
    
#     print(sortedwords)
    
#     print(len(sortedwords[-1]))
    # return the result
    return sortedwords[-2]

In [9]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    
    # convert the dataframe to list for the following processing 
    texts = X_train.T.tolist()

    # cv train using X_train 
    vect = CountVectorizer().fit(X_train)
    
    # transform the texts in the training data to a document-term matrix
    X_train_vectorized = vect.transform(X_train)

    # Train the model
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)

    # Predict the transformed test documents
    predictions = model.predict(vect.transform(X_test))

    return roc_auc_score(y_test, predictions)

In [11]:
answer_three()

0.9720812182741116

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

def answer_four():
    
    # Fit the TfidfVectorizer to the training data
    vect = TfidfVectorizer().fit(X_train)
    
    # transform the texts in the training data to a document-term matrix
    X_train_vectorized = vect.transform(X_train)

    # find the tfidf value and order the tf_idf_index by importance  
    values =  X_train_vectorized.max(0).toarray()[0]
    index = vect.get_feature_names()
    
    # convert the list to the Series required
    features_series = pd.Series(values,index = index)
    
    return features_series.nsmallest(20),features_series.nlargest(20)

In [13]:
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 afternoons      0.091250
 approaching     0.091250
 dtype: float64, 146tf150p    1.000000
 645          1.000000
 anything     1.000000
 anytime      1.000000
 beerage      1.000000
 done         1.000000
 er           1.000000
 havent       1.000000
 home         1.000000
 lei          1.000000
 nite         1.000000
 ok           1.000000
 okie         1.000000
 thank        1.000000
 thanx        1.000000
 too          1.000000
 where        1.000000
 yup          1.000000
 tick         0.980166
 blank        0.932702
 dty

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [0]:
def answer_five():
    
    # Fit the TfidfVectorizer to the training data
    vect = TfidfVectorizer(min_df=3).fit(X_train)
    
    # transform the texts in the training data to a document-term matrix
    X_train_vectorized = vect.transform(X_train)

    # Train the model
    model = MultinomialNB(alpha=0.1)
    model.fit(X_train_vectorized, y_train)

    # Predict the transformed test documents
    predictions = model.predict(vect.transform(X_test))

    return roc_auc_score(y_test, predictions)

In [15]:
answer_five()

0.9416243654822335

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [0]:
def answer_six():
    
    # define spam and not spam text
    spam_texts = spam_data[spam_data['target'] == 1]['text']
    not_spam_texts = spam_data[spam_data['target'] == 0]['text']
    
    # get the spam and not spam length list 
    spam_len_list = [len(length) for length in spam_texts]
    not_spam_len_list = [len(length) for length in not_spam_texts]
    
    # get the average length of spam and not spam texts
    spam_len_avg = sum(spam_len_list)/len(spam_len_list)
    not_spam_len_avg = sum(not_spam_len_list)/len(not_spam_len_list)
     
    return (not_spam_len_avg, spam_len_avg)

In [17]:
answer_six()

(71.02362694300518, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [0]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [0]:
from sklearn.svm import SVC

def answer_seven():
    
    # Fit the TfidfVectorizer to the training data
    vect = TfidfVectorizer(min_df=5).fit(X_train)
    
    # transform the texts in the training data to a document-term matrix
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    # get the len of each X_train as a feature
    X_train_len = X_train.apply(len)
    X_test_len = X_test.apply(len)

    # add the new feature to the original feature matrix
    X_train_aug = add_feature(X_train_vectorized, X_train_len)
    X_test_aug = add_feature(X_test_vectorized, X_test_len)

    # Train the model
    model = SVC(C = 10000)
    model.fit(X_train_aug, y_train)

    # Predict the transformed test documents
    predictions = model.predict(X_test_aug)

    return roc_auc_score(y_test, predictions)

In [20]:
answer_seven()

0.9581366823421557

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [0]:
def answer_eight():
    
    spam_digits_avg = spam_data.loc[spam_data['target'] == 1,'text'].str.count('\d').mean()
    not_spam_digits_avg = spam_data.loc[spam_data['target'] == 0,'text'].str.count('\d').mean()
    
    return (not_spam_digits_avg, spam_digits_avg)

In [22]:
answer_eight()

(0.2992746113989637, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [0]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    
    # Fit the TfidfVectorizer to the training data
    vect = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(X_train)
    
    # transform the texts in the training data to a document-term matrix
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    # get the len of each X_train as a feature
    X_train_len = X_train.apply(len)
    X_test_len = X_test.apply(len)
    
    # find the number of digits per document
    X_train_digits_len = X_train.str.count('\d')
    X_test_digits_len = X_test.str.count('\d')
    
    # add the new feature to the original feature matrix
    X_train_aug_1 = add_feature(X_train_vectorized, X_train_len)
    X_train_aug_2 = add_feature(X_train_aug_1, X_train_digits_len)
    
    X_test_aug_1 = add_feature(X_test_vectorized, X_test_len)
    X_test_aug_2 = add_feature(X_test_aug_1, X_test_digits_len)

    # Train the model
    model = LogisticRegression(C = 100)
    model.fit(X_train_aug_2, y_train)

    # Predict the transformed test documents
    predictions = model.predict(X_test_aug_2)

    return roc_auc_score(y_test, predictions)

In [24]:
answer_nine()

0.9653328353394565

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [0]:
def answer_ten():
    
    spam_char_avg = spam_data.loc[spam_data['target'] == 1,'text'].str.count('\W').mean()
    
    not_spam_char_avg = spam_data.loc[spam_data['target'] == 0,'text'].str.count('\W').mean()
    
    return (not_spam_char_avg, spam_char_avg)

In [26]:
answer_ten()

(17.29181347150259, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [0]:
def answer_eleven():
    
    # Fit the TfidfVectorizer to the training data
    vect = TfidfVectorizer(min_df=5, analyzer='char_wb', ngram_range=(2,5)).fit(X_train)
    
    # transform the texts in the training data to a document-term matrix
    X_train_vectorized = vect.transform(X_train)
    X_test_vectorized = vect.transform(X_test)
    
    # get the len of each X_train as a feature
    X_train_len = X_train.apply(len)
    X_test_len = X_test.apply(len)
    
    # find the number of digits per document
    X_train_digits_len = X_train.str.count('\d')
    X_test_digits_len = X_test.str.count('\d')
    
    # find the average number of non-word characters
    X_train_spam_char_len = X_train.str.count('\W')
    X_test_spam_char_len = X_test.str.count('\W')
    
    
    # add the new feature to the original feature matrix
    X_train_aug_1 = add_feature(X_train_vectorized, X_train_len)
    X_train_aug_2 = add_feature(X_train_aug_1, X_train_digits_len)
    X_train_aug_3 = add_feature(X_train_aug_2, X_train_spam_char_len)
    
    X_test_aug_1 = add_feature(X_test_vectorized, X_test_len)
    X_test_aug_2 = add_feature(X_test_aug_1, X_test_digits_len)
    X_test_aug_3 = add_feature(X_test_aug_2, X_test_spam_char_len)

    # Train the model
    model = LogisticRegression(C = 100)
    model.fit(X_train_aug_3, y_train)

    # Predict the transformed test documents
    predictions = model.predict(X_test_aug_3)

    # get feature names from the vect while adding the new feature names
    feature_names = np.array(vect.get_feature_names() + ["length_of_doc", "digit_count", "non_word_char_count"])

    # sort the importance of the features
    sorted_coef_index = model.coef_[0].argsort()

    # find the most and least important features
    smallest_coef_list = feature_names[sorted_coef_index[:10]].tolist()
    largest_coef_list = feature_names[sorted_coef_index[:-11:-1]].tolist()

    return (roc_auc_score(y_test, predictions), smallest_coef_list, largest_coef_list)

In [28]:
answer_eleven()

(0.9759031798040846,
 ['i ', '..', ' i', ' m', 'ca', 'if', 'if ', '5,', 'my ', ' 6'],
 ['**', 'ww', 'xt', '***', 'co', 'ne', 'ex', 'uk', '****', 'eply '])